# Параметры и функции

In [1]:
#@title
#Параметры рассылки
!pip install pillow qrcode

#Имя документа в Google Sheets
CLUB_CP_MEMBERS_SHEET = 'Название Google таблицы'
#Отправка только своим - True, всем - False
TEAM_SEND = False
#Мия файла лога
LOG_FILE_NAME = 'mailing.log'
#Путь к google Диску
G_DRIVE_PATH = '/content/gdrive'
#smtp сервер
SMTP_SERVER = "адрес smtp сервера"  
SMTP_PORT =   465 # 587 # Порт


#Адрес и пароль для отправки почты
SENDER_EMAIL = "почта отправителя"
SENDER_PASSWORD = "пароль" 
#Адрес отправителя
FROM = 'адрес от кого'
#Адресат по умолчанию
TO   = "адресат по умолчанию"
#Тема письма
SUBJECT = "Тема письма"

#Соответствие называний колонок пример
#словарь key- название столбца для pandas
#value - название столбца в google sheets
COLUMNS_NAMES = {
     'name':'Имя Фамилия'
    ,'phone':'Номер телефона'
    ,'email':'Email'
    ,'note':'Примечания'
    ,'qrc_status':'QR-код отправлен'
    ,'team':'Свои'
    ,'expire':'Срок действия'
    ,'print':'Надо печатать'
}
#строка для адресации столбцов в GS
ABC = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

def make_text(name):
  html = f"""
  <html>
  <body>
  <p>Добрый день, {name} !</p>

  <p>Текст письма</p>

  </body>
  </html>
  
  """
  return html  

#генерация qr  
import qrcode 
#отправка e-mail smtp
import smtplib 
#формирование тела письма
from email.mime.text import MIMEText 
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
#кодирования информации в 64-разрядный код (6 бит), 
#широко используемый в приложениях электронной почты
from email import encoders
#BeautifulSoup для парсинга html в текст
from bs4 import BeautifulSoup as bs
#для работы с таблицей
import pandas as pd
#для подключения google drive
from google.colab import drive
from google.colab import auth
#для работы с google sheets
import gspread
from oauth2client.client import GoogleCredentials
#для работы с датой
from datetime import datetime

def correct_phone_number(p_number):
  return p_number[-10:]


def make_qrcode(row): 
  filename = "qr_code.png"
  # генерируем qr-код
  img = qrcode.make(f"{row['expire']} {row['name']} {correct_phone_number(row['phone'])}")
  # сохраняем img в файл
  img.save(filename)
  return filename


def make_mail(text_to_send, files_to_send, to=TO, 
               from_email=FROM, subject=SUBJECT):
  # initialize the message we wanna send
  msg = MIMEMultipart("mixed")
  msg["From"] = from_email
  msg["To"] = TO
  msg["Subject"] = subject
  message = MIMEMultipart("alternative")
  html = text_to_send
  text = bs(html, "html.parser").text
  message.attach(MIMEText(text, 'plain'))
  message.attach(MIMEText(html, 'html'))
  msg.attach(message)
  part = MIMEBase("image", "png")
  with open(files_to_send, "rb") as attachment:    
    part.set_payload((attachment).read())
  encoders.encode_base64(part)
  part.add_header('Content-Disposition', 
                   f"attachment; filename={files_to_send}")
  msg.attach(part)
  return msg


def do_mailing(mail_list, team=False, to=TO, sender=SENDER_EMAIL, 
               password=SENDER_PASSWORD, from_email=FROM):
  # initialize the SMTP server
  server = smtplib.SMTP_SSL(SMTP_SERVER, SMTP_PORT)
  # connect to the SMTP server as TLS mode (secure) and send EHLO
  # server.starttls()
  # login to the account using the credentials
  server.login(sender, password)
  log = []
  log_str = ''
  for index, row in mail_list.iterrows():
    if (not team) or row['team'] == 'ДА':
      if row['email'] not in ['','-'] and row['qrc_status'] == '':
        try:
          qr_code_file = make_qrcode(row)
          msg = make_mail(make_text(row['name']), qr_code_file, row['email'])

          # send the email
          server.sendmail(from_email, row['email'].split(','), msg.as_string())
          log_str = (f"{datetime.now()} {index} || {row['name']} || "
                  + f"{row['phone']} || {row['email']} || Email has been sent"
                  )
          print(log_str)
          log.append(log_str)
          row['qrc_status'] = 'ДА'
        except BaseException as err:
          log_str = (f"{datetime.now()} {index} || {row['name']} || "
                  + f"{row['phone']} || {row['email']} ||Err {err}, {type(err)}"
                  )
          print(log_str)
          log.append(log_str)
  # terminate the SMTP session
  server.quit()
  
  with open(LOG_FILE_NAME, 'a') as f:
    f.write(str(log)+'\n')
  return mail_list


def get_keybyval(val,dc=COLUMNS_NAMES):
    """
    key by value in dict
    """    
    for idn, v in dc.items():
        if v == val:
            return(idn)
    return val   

def rename_colbyvals(data,vals):
    """
    rename columns by list of vals
    using columns name dict
    """
    for val in vals:
        data = data.rename(columns={val:get_keybyval(val)})
    return data

def do_mailing_and_log(g_sheets):
  #Открываем таблицу
  sheets = g_sheets.open(CLUB_CP_MAMBERS_SHEET).sheet1
  rows = sheets.get_all_values()  
  mail_list = pd.DataFrame(rows[1:], columns=rows[0])
  mail_list = rename_colbyvals(mail_list,mail_list.columns)  
  mail_list =  do_mailing(mail_list, team=TEAM_SEND)

  #Перенос log файла на google диск
  LOG_PATH = G_DRIVE_PATH+'/MyDrive/'+LOG_FILE_NAME
  %mv $LOG_FILE_NAME $LOG_PATH

  #установка статуса отправки писем в google sheets
  n_count = len(mail_list)+1
  n_columns = len(mail_list.columns)
  qrc_s_ind = 0
  for i in range(n_columns):
    if mail_list.columns[i] == 'qrc_status':
      qrc_s_ind = i
  c_let = ABC[qrc_s_ind]

  range_s = f"{c_let}2:{c_let}{str(n_count)}"
  qrc_status = sheets.range(range_s)
  for ind, row in mail_list.iterrows():
    qrc_status[ind].value = row['qrc_status']
  sheets.update_cells(qrc_status)

#подключение Google диска
drive.mount(G_DRIVE_PATH)
#Аутентификация Google аккаунта
auth.authenticate_user()
#Доступ к аккаунту дял библиотеки gspread
g_sheets = gspread.authorize(GoogleCredentials.get_application_default())

     |████████████████████████████████| 43 kB 907 kB/s 
  Created wheel for qrcode: filename=qrcode-7.3.1-py3-none-any.whl size=40402 sha256=32e4ba097ff8360f5e6ccaaf368529cf5bd0c2868c0252d6a347f2127d1fe0cb
  Stored in directory: /root/.cache/pip/wheels/93/d7/39/a4111be2cfb8e679938aa671a37888b6afb1f9e7d748e94492
Successfully built qrcode
Mounted at /content/gdrive


# Рассылка QR кодов

In [ ]:
#@title
do_mailing_and_log(g_sheets)